# Coding

https://github.com/laxmimerit/preprocess_kgptalkie

In [ ]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets
!pip install -U bertviz
!pip install -U umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 41.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Usi

## Data Loading and Preprocessing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import preprocess_kgptalkie as ps

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/shameeha24/shameeha/main/medical_balanced.csv?token=GHSAT0AAAAAACWKQOWXJKBGFOQVBWB7UKLWZWIBUVQ")
#df = df.sample(10_000)
df.head()

,classes,filtered_text
0,4,myocardial sulfhydryl pool alteration occur re...
1,4,silent cerebrovascular disease elderly hyperte...
2,4,fetal maternal response intravenous infusion t...
3,4,role prostaglandin e reducing pulmonary vascul...
4,4,surgical treatment atrial fibrillation develop...


In [ ]:
df.shape


(24025, 2)

In [ ]:
df.isnull().sum()

,0
classes,0
filtered_text,0


# New Section

# preprocessing
df['word_counts'] = df['transcription'].apply(lambda x: ps.get_wordcounts(x))
df['char_counts'] = df['transcription'].apply(lambda x: ps.get_charcounts(x))
df['avg_wordlength'] = df['transcription'].apply(lambda x: ps.get_avg_wordlength(x))
df['stopwords_counts'] = df['transcription'].apply(lambda x: ps.get_stopwords_counts(x))



df['transcription'] =df['transcription'].str.lower()
df['transcription'] = df['transcription'].apply(lambda x: ps.remove_html_tags(x))

/usr/local/lib/python3.10/dist-packages/preprocess_kgptalkie/utils.py:97: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(x, 'lxml').get_text().strip()


## Data Preparation for ML

In [ ]:
# custom dataset -> evaluation/compute metrics -> training arguments -> trainer -> training -> testing

In [ ]:
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

X = df['transcription'].tolist()

label2id = {'neoplasms':1,'digestive disease':2,'nervous disease':3,'cardiovascular disease':4,'pathological conditions':5}
id2label = {1: 'neoplasms',2: 'digestive disease',3: 'nervous disease',4: 'cardiovascular disease',5: 'pathological conditions'}

y = df['classes'].map(label2id).tolist()

dataset = CustomDataset(X, y, tokenizer)

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_len=512):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text = str(self.texts[idx])
    #label = torch.tensor(self.labels[idx]-1)
    label = torch.tensor(self.labels[idx] - 1, dtype=torch.long)  # Adjust labels to be zero-based
    encoding = self.tokenizer(text, truncation=True, padding="max_length",
                              max_length=self.max_len)

    return {
        'input_ids': encoding['input_ids'],
        'attention_mask': encoding['attention_mask'],
        'labels': label
    }

In [ ]:
# Example data
X = df['filtered_text'].tolist()

# Extract labels from DataFrame and convert to list
y = df['classes'].tolist()





In [ ]:
X[1]

'correlation dimension heartbeat interval reduce conscious pig myocardial ischemia correlation dimension heartbeat interval reduce conscious pig myocardial ischemia'

In [ ]:
y[1]

4

In [ ]:
# Check label values before passing to dataset
print("Unique labels before passing to dataset:", set(y))


Unique labels before passing to dataset: {1, 2, 3, 4, 5}


In [ ]:
# Prepare tokenizer and model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=5)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
dataset = CustomDataset(X, y, tokenizer)

In [ ]:
# Split the dataset
from sklearn.model_selection import train_test_split
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# Metrics computation
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(p):
    labels = p.label_ids
    preds = p.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, "f1": f1}




In [ ]:
# Trainer
from transformers import Trainer, TrainingArguments

batch_size = 16
model_name = "distilbert_finetuned_medical"

args = TrainingArguments(
    output_dir=model_name,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=2e-5,
    num_train_epochs=1,
    evaluation_strategy='epoch'
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.179300,1.035168,0.595000,0.585388


TrainOutput(global_step=500, training_loss=1.1792591552734375, metrics={'train_runtime': 397.756, 'train_samples_per_second': 20.113, 'train_steps_per_second': 1.257, 'total_flos': 1059795886080000.0, 'train_loss': 1.1792591552734375, 'epoch': 1.0})

In [ ]:
# Trainer
from transformers import Trainer, TrainingArguments

batch_size = 32
model_name = "distilbert_finetuned_medical"

args = TrainingArguments(
    output_dir=model_name,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=2e-5,
    num_train_epochs=5,
    evaluation_strategy='epoch'
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.004714,0.600500,0.589749
2,0.931100,0.996810,0.598000,0.585457
3,0.931100,1.013656,0.593000,0.584885
4,0.745600,1.045415,0.589000,0.586810
5,0.745600,1.060276,0.584500,0.580164


TrainOutput(global_step=1250, training_loss=0.801674169921875, metrics={'train_runtime': 2039.9321, 'train_samples_per_second': 19.608, 'train_steps_per_second': 0.613, 'total_flos': 5298979430400000.0, 'train_loss': 0.801674169921875, 'epoch': 5.0})

In [ ]:
trainer.save_model(model_name)

## Model Testing

In [ ]:
 id2label = {1: 'neoplasms',2: 'digestive disease',3: 'nervous disease',4: 'cardiovascular disease',5: 'pathological conditions'}

In [ ]:
id2label

{1: 'neoplasms',
 2: 'digestive disease',
 3: 'nervous disease',
 4: 'cardiovascular disease',
 5: 'pathological conditions'}

In [ ]:


tok = AutoTokenizer.from_pretrained(model_name)
mod = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
def get_prediction(text):
  input_ids = tok.encode(text, return_tensors='pt')
  output = mod(input_ids)

  preds = torch.nn.functional.softmax(output.logits, dim=-1)

  prob = torch.max(preds).item()

  idx = torch.argmax(preds).item() + 1  # + 1 is added since labels are from 1. torch.argmax returns value from 0 which could lead to keyerror since 0 label is not present
  disease = id2label[idx]

  return {'disease class':disease, 'prob':prob}

In [ ]:
text = "coexist thymic carcinoid tumor thymoma coexist thymic carcinoid tumor thymoma"
get_prediction(text)

{'disease class': 'neoplasms', 'prob': 0.915410041809082}

In [ ]:
text = "heart rate variability myocardial infarction conscious dog high low risk sudden death heart rate variability myocardial infarction conscious dog high low risk sudden death"
get_prediction(text)

{'disease class': 'cardiovascular disease', 'prob': 0.8043131232261658}

In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, AdamW, get_linear_schedule_with_warmup,EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score



In [ ]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = torch.tensor(self.labels[idx] - 1, dtype=torch.long)  # Adjust labels to be zero-based
        encoding = self.tokenizer(text, truncation=True, padding="max_length", max_length=self.max_len)
        return {
            'input_ids': torch.tensor(encoding['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(encoding['attention_mask'], dtype=torch.long),
            'labels': label
        }



In [ ]:

# Example data
X = df['filtered_text'].tolist()
y = df['classes'].tolist()

# Prepare tokenizer and model
checkpoint = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=5)

# Fine-tune all layers
for param in model.parameters():
    param.requires_grad = True

# Create dataset
dataset = CustomDataset(X, y, tokenizer)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Split the dataset
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# Metrics computation
def compute_metrics(p):
    labels = p.label_ids
    preds = p.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, "f1": f1}



In [ ]:
# Training arguments with added warmup steps and weight decay
batch_size = 16
model_name = "distilbert_finetuned_medical"

# args = TrainingArguments(
#     output_dir=model_name,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     learning_rate=3e-5,  # Experiment with different learning rates
#     num_train_epochs=5,  # Increase the number of epochs
#     warmup_steps=500,  # Add warmup steps
#     weight_decay=0.01,  # Add weight decay
#     evaluation_strategy='epoch',
#     save_total_limit=2,
#     load_best_model_at_end=True
# )


# Trainer
# trainer = Trainer(
#     model=model,
#     args=args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     compute_metrics=compute_metrics,
#     tokenizer=tokenizer,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
# )



In [ ]:
# Training arguments with early stopping and learning rate scheduler
args = TrainingArguments(
    output_dir=model_name,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=1e-5,
    num_train_epochs=10,  # Allow for early stopping
    evaluation_strategy='epoch',  # Evaluate at the end of each epoch
    save_strategy='epoch',  # Save at the end of each epoch
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_dir='./logs',
    logging_steps=10,
    warmup_steps=500,  # Add warmup steps
    weight_decay=0.01,  # Add weight decay
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Trainer with early stopping callback
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Stop training if no improvement after 3 evaluations
)

In [ ]:

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.184100,1.119650,0.562000,0.542877
2,0.970100,1.007001,0.606000,0.593750
3,0.920200,0.992602,0.611500,0.604764
4,0.823800,1.018227,0.592000,0.590202
5,0.761800,1.035200,0.599500,0.592315
6,0.624100,1.082050,0.587500,0.582224


TrainOutput(global_step=3000, training_loss=0.940635630607605, metrics={'train_runtime': 2350.7888, 'train_samples_per_second': 34.031, 'train_steps_per_second': 2.127, 'total_flos': 6358775316480000.0, 'train_loss': 0.940635630607605, 'epoch': 6.0})

In [ ]:
# Training arguments with early stopping and learning rate scheduler
args = TrainingArguments(
    output_dir=model_name,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=3e-5,
    num_train_epochs=1,  # Allow for early stopping
    evaluation_strategy='epoch',  # Evaluate at the end of each epoch
    save_strategy='epoch',  # Save at the end of each epoch
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_dir='./logs',
    logging_steps=10,
    warmup_steps=500,  # Add warmup steps
    weight_decay=0.01,  # Add weight decay
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Trainer with early stopping callback
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Stop training if no improvement after 3 evaluations
)

In [ ]:

# Train the model
trainer.train()

NameError: name 'trainer' is not defined

# training with medical_balanced data

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/shameeha24/shameeha/main/medical_balanced.csv?token=GHSAT0AAAAAACWKQOWXZ5MLQWPSYDBCQCICZWEIVMA")
#df = df.sample(10_000)
df.head()

,classes,filtered_text
0,4,myocardial sulfhydryl pool alteration occur re...
1,4,silent cerebrovascular disease elderly hyperte...
2,4,fetal maternal response intravenous infusion t...
3,4,role prostaglandin e reducing pulmonary vascul...
4,4,surgical treatment atrial fibrillation develop...


In [ ]:
df.shape


(24025, 2)

In [ ]:
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, AdamW, get_linear_schedule_with_warmup,EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score



In [ ]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = torch.tensor(self.labels[idx] - 1, dtype=torch.long)  # Adjust labels to be zero-based
        encoding = self.tokenizer(text, truncation=True, padding="max_length", max_length=self.max_len)
        return {
            'input_ids': torch.tensor(encoding['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(encoding['attention_mask'], dtype=torch.long),
            'labels': label
        }



In [ ]:

# Example data
X = df['filtered_text'].tolist()
y = df['classes'].tolist()

# Prepare tokenizer and model
checkpoint = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=5)

# Fine-tune all layers
for param in model.parameters():
    param.requires_grad = True

# Create dataset
dataset = CustomDataset(X, y, tokenizer)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Split the dataset
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# Metrics computation
def compute_metrics(p):
    labels = p.label_ids
    preds = p.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, "f1": f1}



In [ ]:
# Training arguments with added warmup steps and weight decay
batch_size = 16
model_name = "distilbert_finetuned_medical"

# args = TrainingArguments(
#     output_dir=model_name,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     learning_rate=3e-5,  # Experiment with different learning rates
#     num_train_epochs=5,  # Increase the number of epochs
#     warmup_steps=500,  # Add warmup steps
#     weight_decay=0.01,  # Add weight decay
#     evaluation_strategy='epoch',
#     save_total_limit=2,
#     load_best_model_at_end=True
# )


# Trainer
# trainer = Trainer(
#     model=model,
#     args=args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     compute_metrics=compute_metrics,
#     tokenizer=tokenizer,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
# )



In [ ]:
# Training arguments with early stopping and learning rate scheduler
args = TrainingArguments(
    output_dir=model_name,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=1e-5,
    num_train_epochs=1,  # Allow for early stopping
    evaluation_strategy='epoch',  # Evaluate at the end of each epoch
    save_strategy='epoch',  # Save at the end of each epoch
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_dir='./logs',
    logging_steps=10,
    warmup_steps=500,  # Add warmup steps
    weight_decay=0.01,  # Add weight decay
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Trainer with early stopping callback
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Stop training if no improvement after 3 evaluations
)

In [ ]:

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.954600,0.885996,0.682206,0.660794


TrainOutput(global_step=1202, training_loss=1.1741943668803438, metrics={'train_runtime': 914.1412, 'train_samples_per_second': 21.025, 'train_steps_per_second': 1.315, 'total_flos': 2546159616307200.0, 'train_loss': 1.1741943668803438, 'epoch': 1.0})

**learning rate = 2e-5**

In [ ]:
# Training arguments with early stopping and learning rate scheduler
args = TrainingArguments(
    output_dir=model_name,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=2e-5,
    num_train_epochs=5,  # Allow for early stopping
    evaluation_strategy='epoch',  # Evaluate at the end of each epoch
    save_strategy='epoch',  # Save at the end of each epoch
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_dir='./logs',
    logging_steps=10,
    warmup_steps=500,  # Add warmup steps
    weight_decay=0.01,  # Add weight decay
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Trainer with early stopping callback
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stop training if no improvement after 3 evaluations
)

In [ ]:

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.867800,0.812073,0.704891,0.687479
2,0.630400,0.730067,0.742144,0.729744
3,0.767400,0.661123,0.765036,0.755366


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.867800,0.812073,0.704891,0.687479
2,0.630400,0.730067,0.742144,0.729744
3,0.767400,0.661123,0.765036,0.755366
4,0.365600,0.675605,0.770864,0.764745
5,0.486400,0.676888,0.778772,0.773188


TrainOutput(global_step=6010, training_loss=0.6583971605523057, metrics={'train_runtime': 4700.5556, 'train_samples_per_second': 20.444, 'train_steps_per_second': 1.279, 'total_flos': 1.2730798081536e+16, 'train_loss': 0.6583971605523057, 'epoch': 5.0})

# medical_balanced_avg data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/Minni-N/Minni/main/medical_balanced_avg.csv?token=GHSAT0AAAAAACSNWNTWU6KCGAROT7OLUZ4EZSPL3BA")
#df = df.sample(10_000)
df.head()

,classes,filtered_text
0,4,ecmo assisted angioplasty cardiomyopathy patie...
1,4,congestive heart failure coronary event athero...
2,4,giant carotidophthalmic artery aneurysm direct...
3,4,longterm treatment hereditary angioedema atten...
4,4,endovascular management arterial intimal defec...


In [ ]:
df.shape


(14435, 2)

In [ ]:
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, AdamW, get_linear_schedule_with_warmup,EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score



In [ ]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = torch.tensor(self.labels[idx] - 1, dtype=torch.long)  # Adjust labels to be zero-based
        encoding = self.tokenizer(text, truncation=True, padding="max_length", max_length=self.max_len)
        return {
            'input_ids': torch.tensor(encoding['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(encoding['attention_mask'], dtype=torch.long),
            'labels': label
        }



In [ ]:

# Example data
X = df['filtered_text'].tolist()
y = df['classes'].tolist()

# Prepare tokenizer and model
checkpoint = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=5)

# Fine-tune all layers
for param in model.parameters():
    param.requires_grad = True

# Create dataset
dataset = CustomDataset(X, y, tokenizer)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Split the dataset
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# Metrics computation
def compute_metrics(p):
    labels = p.label_ids
    preds = p.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, "f1": f1}



In [ ]:
# Training arguments with added warmup steps and weight decay
batch_size = 16
model_name = "distilbert_finetuned_medical"

# args = TrainingArguments(
#     output_dir=model_name,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     learning_rate=3e-5,  # Experiment with different learning rates
#     num_train_epochs=5,  # Increase the number of epochs
#     warmup_steps=500,  # Add warmup steps
#     weight_decay=0.01,  # Add weight decay
#     evaluation_strategy='epoch',
#     save_total_limit=2,
#     load_best_model_at_end=True
# )


# Trainer
# trainer = Trainer(
#     model=model,
#     args=args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     compute_metrics=compute_metrics,
#     tokenizer=tokenizer,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
# )



In [ ]:
# Training arguments with early stopping and learning rate scheduler
args = TrainingArguments(
    output_dir=model_name,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=3e-5,
    num_train_epochs=7,  # Allow for early stopping
    evaluation_strategy='epoch',  # Evaluate at the end of each epoch
    save_strategy='epoch',  # Save at the end of each epoch
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_dir='./logs',
    logging_steps=10,
    warmup_steps=500,  # Add warmup steps
    weight_decay=0.01,  # Add weight decay
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Trainer with early stopping callback
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stop training if no improvement after 3 evaluations
)

In [ ]:

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.843500,0.790912,0.713424,0.696385
2,0.637900,0.718376,0.743184,0.732372
3,0.710500,0.654249,0.766077,0.756239
4,0.354600,0.661850,0.778980,0.768389
5,0.461500,0.667486,0.776691,0.769343


TrainOutput(global_step=6010, training_loss=0.6341310514983243, metrics={'train_runtime': 4690.8974, 'train_samples_per_second': 28.681, 'train_steps_per_second': 1.794, 'total_flos': 1.2730798081536e+16, 'train_loss': 0.6341310514983243, 'epoch': 5.0})